In [ ]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00


#qubit1 for BigBasket product

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 1
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0
Iteration 1/50, Best Rating: 5.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best Rating: 5.0
Iteration 31/50, Best 

qubit2_BBProduct

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 4.2
Iteration 2/50, Best Rating: 4.550000000000001
Iteration 3/50, Best Rating: 4.550000000000001
Iteration 4/50, Best Rating: 4.550000000000001
Iteration 5/50, Best Rating: 4.550000000000001
Iteration 6/50, Best Rating: 4.550000000000001
Iteration 7/50, Best Rating: 4.550000000000001
Iteration 8/50, Best Rating: 4.550000000000001
Iteration 9/50, Best Rating: 4.550000000000001
Iteration 10/50, Best Rating: 4.550000000000001
Iteration 11/50, Best Rating: 4.65
Iteration 12/50, Best Rating: 4.65
Iteration 13/50, Best Rating: 4.65
Iteration 14/50, Best Rating: 4.65
Iteration 15/50, Best Rating: 4.65
Iteration 16/50, Best Rating: 4.65
Iteration 17/50, Best Rating: 4.65
Iteration 18/50, Best Rating: 4.65
Iteration 19/50, Best Rating: 4.65
Iteration 20/50, Best Rating: 4.65
Iteration 21/50, Best Rating: 4.65
Iteration 22/50, Best Rating: 4.65
Iteration 23/50, Best Rating: 4.8
Iteration 24/50, Best Rating: 4.8
Iteration 25/50, Best Rating: 4.8
Iteration 26/50, Best

#qbit3_bbmarket*

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 3
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 4.166666666666666
Iteration 2/50, Best Rating: 4.3999999999999995
Iteration 3/50, Best Rating: 4.3999999999999995
Iteration 4/50, Best Rating: 4.3999999999999995
Iteration 5/50, Best Rating: 4.3999999999999995
Iteration 6/50, Best Rating: 4.4
Iteration 7/50, Best Rating: 4.4
Iteration 8/50, Best Rating: 4.4
Iteration 9/50, Best Rating: 4.433333333333334
Iteration 10/50, Best Rating: 4.433333333333334
Iteration 11/50, Best Rating: 4.433333333333334
Iteration 12/50, Best Rating: 4.5
Iteration 13/50, Best Rating: 4.5
Iteration 14/50, Best Rating: 4.5
Iteration 15/50, Best Rating: 4.5
Iteration 16/50, Best Rating: 4.5
Iteration 17/50, Best Rating: 4.5
Iteration 18/50, Best Rating: 4.5
Iteration 19/50, Best Rating: 4.5
Iteration 20/50, Best Rating: 4.5
Iteration 21/50, Best Rating: 4.5
Iteration 22/50, Best Rating: 4.5
Iteration 23/50, Best Rating: 4.5
Iteration 24/50, Best Rating: 4.6000000000000005
Iteration 25/50, Best Rating: 4.6000000000000005
Iteration 26/

#qbit4

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 3.5799999999999996
Iteration 2/50, Best Rating: 4.225
Iteration 3/50, Best Rating: 4.225
Iteration 4/50, Best Rating: 4.225
Iteration 5/50, Best Rating: 4.225
Iteration 6/50, Best Rating: 4.225
Iteration 7/50, Best Rating: 4.225
Iteration 8/50, Best Rating: 4.225
Iteration 9/50, Best Rating: 4.525
Iteration 10/50, Best Rating: 4.525
Iteration 11/50, Best Rating: 4.525
Iteration 12/50, Best Rating: 4.525
Iteration 13/50, Best Rating: 4.525
Iteration 14/50, Best Rating: 4.525
Iteration 15/50, Best Rating: 4.525
Iteration 16/50, Best Rating: 4.525
Iteration 17/50, Best Rating: 4.525
Iteration 18/50, Best Rating: 4.525
Iteration 19/50, Best Rating: 4.525
Iteration 20/50, Best Rating: 4.525
Iteration 21/50, Best Rating: 4.525
Iteration 22/50, Best Rating: 4.525
Iteration 23/50, Best Rating: 4.525
Iteration 24/50, Best Rating: 4.525
Iteration 25/50, Best Rating: 4.525
Iteration 26/50, Best Rating: 4.525
Iteration 27/50, Best Rating: 4.525
Iteration 28/50, Best Ra

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

     # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    # Apply controlled-Not gates for entanglement
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 4.26
Iteration 2/50, Best Rating: 4.4
Iteration 3/50, Best Rating: 4.4
Iteration 4/50, Best Rating: 4.4
Iteration 5/50, Best Rating: 4.4
Iteration 6/50, Best Rating: 4.4
Iteration 7/50, Best Rating: 4.4
Iteration 8/50, Best Rating: 4.4
Iteration 9/50, Best Rating: 4.4
Iteration 10/50, Best Rating: 4.4
Iteration 11/50, Best Rating: 4.4
Iteration 12/50, Best Rating: 4.4
Iteration 13/50, Best Rating: 4.4
Iteration 14/50, Best Rating: 4.4
Iteration 15/50, Best Rating: 4.4
Iteration 16/50, Best Rating: 4.4
Iteration 17/50, Best Rating: 4.55
Iteration 18/50, Best Rating: 4.7
Iteration 19/50, Best Rating: 4.7
Iteration 20/50, Best Rating: 4.7
Iteration 21/50, Best Rating: 4.7
Iteration 22/50, Best Rating: 4.7
Iteration 23/50, Best Rating: 4.7
Iteration 24/50, Best Rating: 4.7
Iteration 25/50, Best Rating: 4.7
Iteration 26/50, Best Rating: 4.7
Iteration 27/50, Best Rating: 4.7
Iteration 28/50, Best Rating: 4.7
Iteration 29/50, Best Rating: 4.7
Iteration 30/50, Best

qbit5

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 5
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 4.08
Iteration 2/50, Best Rating: 4.32
Iteration 3/50, Best Rating: 4.32
Iteration 4/50, Best Rating: 4.32
Iteration 5/50, Best Rating: 4.32
Iteration 6/50, Best Rating: 4.32
Iteration 7/50, Best Rating: 4.32
Iteration 8/50, Best Rating: 4.34
Iteration 9/50, Best Rating: 4.34
Iteration 10/50, Best Rating: 4.42
Iteration 11/50, Best Rating: 4.42
Iteration 12/50, Best Rating: 4.42
Iteration 13/50, Best Rating: 4.42
Iteration 14/50, Best Rating: 4.42
Iteration 15/50, Best Rating: 4.42
Iteration 16/50, Best Rating: 4.42
Iteration 17/50, Best Rating: 4.42
Iteration 18/50, Best Rating: 4.42
Iteration 19/50, Best Rating: 4.42
Iteration 20/50, Best Rating: 4.42
Iteration 21/50, Best Rating: 4.44
Iteration 22/50, Best Rating: 4.44
Iteration 23/50, Best Rating: 4.44
Iteration 24/50, Best Rating: 4.44
Iteration 25/50, Best Rating: 4.44
Iteration 26/50, Best Rating: 4.44
Iteration 27/50, Best Rating: 4.44
Iteration 28/50, Best Rating: 4.44
Iteration 29/50, Best Rating:

#qbit6

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 6
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 4.279999999999999
Iteration 2/50, Best Rating: 4.279999999999999
Iteration 3/50, Best Rating: 4.279999999999999
Iteration 4/50, Best Rating: 4.279999999999999
Iteration 5/50, Best Rating: 4.279999999999999
Iteration 6/50, Best Rating: 4.279999999999999
Iteration 7/50, Best Rating: 4.62
Iteration 8/50, Best Rating: 4.62
Iteration 9/50, Best Rating: 4.62
Iteration 10/50, Best Rating: 4.62
Iteration 11/50, Best Rating: 4.62
Iteration 12/50, Best Rating: 4.62
Iteration 13/50, Best Rating: 4.62
Iteration 14/50, Best Rating: 4.62
Iteration 15/50, Best Rating: 4.62
Iteration 16/50, Best Rating: 4.62
Iteration 17/50, Best Rating: 4.62
Iteration 18/50, Best Rating: 4.62
Iteration 19/50, Best Rating: 4.62
Iteration 20/50, Best Rating: 4.62
Iteration 21/50, Best Rating: 4.62
Iteration 22/50, Best Rating: 4.62
Iteration 23/50, Best Rating: 4.62
Iteration 24/50, Best Rating: 4.62
Iteration 25/50, Best Rating: 4.62
Iteration 26/50, Best Rating: 4.62
Iteration 27/50, Best

#qbit7

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('BigBasket Products.csv')

# Convert sale_price to numeric if necessary
df['sale_price'] = pd.to_numeric(df['sale_price'], errors='coerce')

# Ensure rating is numeric
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['sale_price', 'rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['sale_price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with 7 qubits
n_qubits = 7
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['rating'] >= user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product']}, Rating: {row['rating']}, Sale Price: {row['sale_price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))


Please enter the minimum rating (0-10): 4.9
Top 5 Recommended Products:
Product: Face Wash - Oil Control, Active, Rating: 5.0, Sale Price: 110.0
Product: Original Eau De Toilette, Rating: 5.0, Sale Price: 1837.5
Product: Gourmets Delite  Wafers - Mango 75 gm Pouch, Rating: 5.0, Sale Price: 45.0
Product: Premium Soft Diaper Pants - Extra Small, Rating: 5.0, Sale Price: 1091.09
Product: Adult Dry Cat Food - +1 Year, Ocean Fish, Rating: 5.0, Sale Price: 429.0


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 4.26
Iteration 2/50, Best Rating: 4.5
Iteration 3/50, Best Rating: 4.7
Iteration 4/50, Best Rating: 4.7
Iteration 5/50, Best Rating: 4.7
Iteration 6/50, Best Rating: 4.7
Iteration 7/50, Best Rating: 4.7
Iteration 8/50, Best Rating: 4.7
Iteration 9/50, Best Rating: 4.7
Iteration 10/50, Best Rating: 4.7
Iteration 11/50, Best Rating: 4.7
Iteration 12/50, Best Rating: 4.7
Iteration 13/50, Best Rating: 4.82
Iteration 14/50, Best Rating: 4.82
Iteration 15/50, Best Rating: 4.82
Iteration 16/50, Best Rating: 4.82
Iteration 17/50, Best Rating: 4.82
Iteration 18/50, Best Rating: 4.82
Iteration 19/50, Best Rating: 4.82
Iteration 20/50, Best Rating: 4.82
Iteration 21/50, Best Rating: 4.82
Iteration 22/50, Best Rating: 4.82
Iteration 23/50, Best Rating: 4.82
Iteration 24/50, Best Rating: 4.82
Iteration 25/50, Best Rating: 4.82
Iteration 26/50, Best Rating: 4.82
Iteration 27/50, Best Rating: 4.82
Iteration 28/50, Best Rating: 4.82
Iteration 29/50, Best Rating: 4.82
Itera

#qubit1_supermarket sales

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/supermarket_sales - Sheet1.csv')

# Convert sale_price to numeric if necessary
df['Unit price'] = pd.to_numeric(df['Unit price'], errors='coerce')

# Ensure rating is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['Unit price', 'Rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['Unit price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with n qubits
n_qubits = 1
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['Rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['Rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['Rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['Rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='Rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='Rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 8.0
Top 5 Recommended Products:
Product: Home and lifestyle, Rating: 8.0, Sale Price: 73.56
Product: Fashion accessories, Rating: 8.0, Sale Price: 92.98
Product: Fashion accessories, Rating: 8.0, Sale Price: 48.96
Product: Food and beverages, Rating: 8.0, Sale Price: 28.86
Product: Health and beauty, Rating: 8.0, Sale Price: 85.87


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 9.8
Iteration 2/50, Best Rating: 9.8
Iteration 3/50, Best Rating: 9.8
Iteration 4/50, Best Rating: 10.0
Iteration 5/50, Best Rating: 10.0
Iteration 6/50, Best Rating: 10.0
Iteration 7/50, Best Rating: 10.0
Iteration 8/50, Best Rating: 10.0
Iteration 9/50, Best Rating: 10.0
Iteration 10/50, Best Rating: 10.0
Iteration 11/50, Best Rating: 10.0
Iteration 12/50, Best Rating: 10.0
Iteration 13/50, Best Rating: 10.0
Iteration 14/50, Best Rating: 10.0
Iteration 15/50, Best Rating: 10.0
Iteration 16/50, Best Rating: 10.0
Iteration 17/50, Best Rating: 10.0
Iteration 18/50, Best Rating: 10.0
Iteration 19/50, Best Rating: 10.0
Iteration 20/50, Best Rating: 10.0
Iteration 21/50, Best Rating: 10.0
Iteration 22/50, Best Rating: 10.0
Iteration 23/50, Best Rating: 10.0
Iteration 24/50, Best Rating: 10.0
Iteration 25/50, Best Rating: 10.0
Iteration 26/50, Best Rating: 10.0
Iteration 27/50, Best Rating: 10.0
Iteration 28/50, Best Rating: 10.0
Iteration 29/50, Best Rating: 10

#qbit2_ssmarket

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/supermarket_sales - Sheet1.csv')

# Convert sale_price to numeric if necessary
df['Unit price'] = pd.to_numeric(df['Unit price'], errors='coerce')

# Ensure rating is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['Unit price', 'Rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['Unit price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with n qubits
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['Rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['Rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['Rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['Rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='Rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='Rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 8.0
Top 5 Recommended Products:
Product: Home and lifestyle, Rating: 8.0, Sale Price: 73.56
Product: Fashion accessories, Rating: 8.0, Sale Price: 92.98
Product: Fashion accessories, Rating: 8.0, Sale Price: 48.96
Product: Food and beverages, Rating: 8.0, Sale Price: 28.86
Product: Health and beauty, Rating: 8.0, Sale Price: 85.87


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 8.440000000000001
Iteration 2/50, Best Rating: 8.440000000000001
Iteration 3/50, Best Rating: 8.440000000000001
Iteration 4/50, Best Rating: 8.440000000000001
Iteration 5/50, Best Rating: 8.65
Iteration 6/50, Best Rating: 8.65
Iteration 7/50, Best Rating: 8.65
Iteration 8/50, Best Rating: 8.850000000000001
Iteration 9/50, Best Rating: 8.98
Iteration 10/50, Best Rating: 8.98
Iteration 11/50, Best Rating: 9.75
Iteration 12/50, Best Rating: 9.75
Iteration 13/50, Best Rating: 9.75
Iteration 14/50, Best Rating: 9.75
Iteration 15/50, Best Rating: 9.75
Iteration 16/50, Best Rating: 9.75
Iteration 17/50, Best Rating: 9.75
Iteration 18/50, Best Rating: 9.75
Iteration 19/50, Best Rating: 9.75
Iteration 20/50, Best Rating: 9.75
Iteration 21/50, Best Rating: 9.75
Iteration 22/50, Best Rating: 9.75
Iteration 23/50, Best Rating: 9.75
Iteration 24/50, Best Rating: 9.75
Iteration 25/50, Best Rating: 9.75
Iteration 26/50, Best Rating: 9.75
Iteration 27/50, Best Rating: 9.75

#qbit3

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/supermarket_sales - Sheet1.csv')

# Convert sale_price to numeric if necessary
df['Unit price'] = pd.to_numeric(df['Unit price'], errors='coerce')

# Ensure rating is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['Unit price', 'Rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['Unit price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with n qubits
n_qubits = 3
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['Rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['Rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['Rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['Rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='Rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='Rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 8.0
Top 5 Recommended Products:
Product: Home and lifestyle, Rating: 8.0, Sale Price: 73.56
Product: Fashion accessories, Rating: 8.0, Sale Price: 92.98
Product: Fashion accessories, Rating: 8.0, Sale Price: 48.96
Product: Food and beverages, Rating: 8.0, Sale Price: 28.86
Product: Health and beauty, Rating: 8.0, Sale Price: 85.87


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 7.94
Iteration 2/50, Best Rating: 8.166666666666666
Iteration 3/50, Best Rating: 8.866666666666667
Iteration 4/50, Best Rating: 8.866666666666667
Iteration 5/50, Best Rating: 8.866666666666667
Iteration 6/50, Best Rating: 8.866666666666667
Iteration 7/50, Best Rating: 8.866666666666667
Iteration 8/50, Best Rating: 8.866666666666667
Iteration 9/50, Best Rating: 8.866666666666667
Iteration 10/50, Best Rating: 8.866666666666667
Iteration 11/50, Best Rating: 9.066666666666668
Iteration 12/50, Best Rating: 9.066666666666668
Iteration 13/50, Best Rating: 9.066666666666668
Iteration 14/50, Best Rating: 9.066666666666668
Iteration 15/50, Best Rating: 9.133333333333333
Iteration 16/50, Best Rating: 9.133333333333333
Iteration 17/50, Best Rating: 9.133333333333333
Iteration 18/50, Best Rating: 9.133333333333333
Iteration 19/50, Best Rating: 9.133333333333333
Iteration 20/50, Best Rating: 9.133333333333333
Iteration 21/50, Best Rating: 9.133333333333333
Iteration 22/5

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/supermarket_sales - Sheet1.csv')

# Convert sale_price to numeric if necessary
df['Unit price'] = pd.to_numeric(df['Unit price'], errors='coerce')

# Ensure rating is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['Unit price', 'Rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['Unit price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with n qubits
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['Rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['Rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['Rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['Rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='Rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='Rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 8.0
Top 5 Recommended Products:
Product: Home and lifestyle, Rating: 8.0, Sale Price: 73.56
Product: Fashion accessories, Rating: 8.0, Sale Price: 92.98
Product: Fashion accessories, Rating: 8.0, Sale Price: 48.96
Product: Food and beverages, Rating: 8.0, Sale Price: 28.86
Product: Health and beauty, Rating: 8.0, Sale Price: 85.87


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 8.625
Iteration 2/50, Best Rating: 8.64
Iteration 3/50, Best Rating: 8.64
Iteration 4/50, Best Rating: 8.780000000000001
Iteration 5/50, Best Rating: 8.780000000000001
Iteration 6/50, Best Rating: 8.780000000000001
Iteration 7/50, Best Rating: 8.780000000000001
Iteration 8/50, Best Rating: 8.780000000000001
Iteration 9/50, Best Rating: 8.780000000000001
Iteration 10/50, Best Rating: 8.780000000000001
Iteration 11/50, Best Rating: 8.780000000000001
Iteration 12/50, Best Rating: 8.780000000000001
Iteration 13/50, Best Rating: 8.780000000000001
Iteration 14/50, Best Rating: 8.780000000000001
Iteration 15/50, Best Rating: 8.780000000000001
Iteration 16/50, Best Rating: 8.780000000000001
Iteration 17/50, Best Rating: 8.780000000000001
Iteration 18/50, Best Rating: 8.780000000000001
Iteration 19/50, Best Rating: 9.3
Iteration 20/50, Best Rating: 9.3
Iteration 21/50, Best Rating: 9.3
Iteration 22/50, Best Rating: 9.3
Iteration 23/50, Best Rating: 9.3
Iteration 24/

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/supermarket_sales - Sheet1.csv')

# Convert sale_price to numeric if necessary
df['Unit price'] = pd.to_numeric(df['Unit price'], errors='coerce')

# Ensure rating is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['Unit price', 'Rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['Unit price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with n qubits
n_qubits = 5
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['Rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['Rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['Rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['Rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='Rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='Rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 8.0
Top 5 Recommended Products:
Product: Home and lifestyle, Rating: 8.0, Sale Price: 73.56
Product: Fashion accessories, Rating: 8.0, Sale Price: 92.98
Product: Fashion accessories, Rating: 8.0, Sale Price: 48.96
Product: Food and beverages, Rating: 8.0, Sale Price: 28.86
Product: Health and beauty, Rating: 8.0, Sale Price: 85.87


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 7.279999999999999
Iteration 2/50, Best Rating: 8.420000000000002
Iteration 3/50, Best Rating: 8.420000000000002
Iteration 4/50, Best Rating: 8.420000000000002
Iteration 5/50, Best Rating: 8.420000000000002
Iteration 6/50, Best Rating: 8.420000000000002
Iteration 7/50, Best Rating: 8.420000000000002
Iteration 8/50, Best Rating: 8.420000000000002
Iteration 9/50, Best Rating: 8.420000000000002
Iteration 10/50, Best Rating: 8.420000000000002
Iteration 11/50, Best Rating: 8.420000000000002
Iteration 12/50, Best Rating: 8.420000000000002
Iteration 13/50, Best Rating: 8.48
Iteration 14/50, Best Rating: 8.48
Iteration 15/50, Best Rating: 8.48
Iteration 16/50, Best Rating: 8.48
Iteration 17/50, Best Rating: 8.48
Iteration 18/50, Best Rating: 8.48
Iteration 19/50, Best Rating: 8.48
Iteration 20/50, Best Rating: 8.8
Iteration 21/50, Best Rating: 8.8
Iteration 22/50, Best Rating: 8.8
Iteration 23/50, Best Rating: 8.8
Iteration 24/50, Best Rating: 8.8
Iteration 25/50, B

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/supermarket_sales - Sheet1.csv')

# Convert sale_price to numeric if necessary
df['Unit price'] = pd.to_numeric(df['Unit price'], errors='coerce')

# Ensure rating is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['Unit price', 'Rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['Unit price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with n qubits
n_qubits = 6
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['Rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['Rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['Rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['Rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='Rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='Rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 8.0
Top 5 Recommended Products:
Product: Home and lifestyle, Rating: 8.0, Sale Price: 73.56
Product: Fashion accessories, Rating: 8.0, Sale Price: 92.98
Product: Fashion accessories, Rating: 8.0, Sale Price: 48.96
Product: Food and beverages, Rating: 8.0, Sale Price: 28.86
Product: Health and beauty, Rating: 8.0, Sale Price: 85.87


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 8.58
Iteration 2/50, Best Rating: 8.58
Iteration 3/50, Best Rating: 8.879999999999999
Iteration 4/50, Best Rating: 8.879999999999999
Iteration 5/50, Best Rating: 8.879999999999999
Iteration 6/50, Best Rating: 8.879999999999999
Iteration 7/50, Best Rating: 8.98
Iteration 8/50, Best Rating: 8.98
Iteration 9/50, Best Rating: 8.98
Iteration 10/50, Best Rating: 8.98
Iteration 11/50, Best Rating: 9.1
Iteration 12/50, Best Rating: 9.1
Iteration 13/50, Best Rating: 9.1
Iteration 14/50, Best Rating: 9.1
Iteration 15/50, Best Rating: 9.1
Iteration 16/50, Best Rating: 9.1
Iteration 17/50, Best Rating: 9.1
Iteration 18/50, Best Rating: 9.1
Iteration 19/50, Best Rating: 9.1
Iteration 20/50, Best Rating: 9.1
Iteration 21/50, Best Rating: 9.24
Iteration 22/50, Best Rating: 9.24
Iteration 23/50, Best Rating: 9.24
Iteration 24/50, Best Rating: 9.24
Iteration 25/50, Best Rating: 9.24
Iteration 26/50, Best Rating: 9.24
Iteration 27/50, Best Rating: 9.24
Iteration 28/50, Best 

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/supermarket_sales - Sheet1.csv')

# Convert sale_price to numeric if necessary
df['Unit price'] = pd.to_numeric(df['Unit price'], errors='coerce')

# Ensure rating is numeric
df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['Unit price', 'Rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['Unit price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization with n qubits
n_qubits = 7
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['Rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['Rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['Rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['Rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='Rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='Rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['Product line']}, Rating: {row['Rating']}, Sale Price: {row['Unit price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 8.0
Top 5 Recommended Products:
Product: Home and lifestyle, Rating: 8.0, Sale Price: 73.56
Product: Fashion accessories, Rating: 8.0, Sale Price: 92.98
Product: Fashion accessories, Rating: 8.0, Sale Price: 48.96
Product: Food and beverages, Rating: 8.0, Sale Price: 28.86
Product: Health and beauty, Rating: 8.0, Sale Price: 85.87


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 8.680000000000001
Iteration 2/50, Best Rating: 9.4
Iteration 3/50, Best Rating: 9.4
Iteration 4/50, Best Rating: 9.66
Iteration 5/50, Best Rating: 9.66
Iteration 6/50, Best Rating: 9.66
Iteration 7/50, Best Rating: 9.66
Iteration 8/50, Best Rating: 9.66
Iteration 9/50, Best Rating: 9.66
Iteration 10/50, Best Rating: 9.66
Iteration 11/50, Best Rating: 9.66
Iteration 12/50, Best Rating: 9.66
Iteration 13/50, Best Rating: 9.66
Iteration 14/50, Best Rating: 9.66
Iteration 15/50, Best Rating: 9.66
Iteration 16/50, Best Rating: 9.66
Iteration 17/50, Best Rating: 9.66
Iteration 18/50, Best Rating: 9.66
Iteration 19/50, Best Rating: 9.66
Iteration 20/50, Best Rating: 9.66
Iteration 21/50, Best Rating: 9.66
Iteration 22/50, Best Rating: 9.66
Iteration 23/50, Best Rating: 9.66
Iteration 24/50, Best Rating: 9.66
Iteration 25/50, Best Rating: 9.66
Iteration 26/50, Best Rating: 9.66
Iteration 27/50, Best Rating: 9.66
Iteration 28/50, Best Rating: 9.66
Iteration 29/50, B

#qbit1_nyka

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/nyka_top_brands_cosmetics_product_reviews.csv')

# Convert sale_price to numeric if necessary
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Ensure rating is numeric
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['price', 'review_rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization
n_qubits = 1
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['review_rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['review_rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['review_rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['review_rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='review_rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='review_rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 4.0
Top 5 Recommended Products:
Product: Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF), Rating: 4.0, Sale Price: 1599
Product: L'Oreal Paris Volume Million Lashes Waterproof Mascara, Rating: 4.0, Sale Price: 999
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 5.0
Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best R

#qbit2

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/nyka_top_brands_cosmetics_product_reviews.csv')

# Convert sale_price to numeric if necessary
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Ensure rating is numeric
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['price', 'review_rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['review_rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['review_rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['review_rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['review_rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='review_rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='review_rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 4.0
Top 5 Recommended Products:
Product: Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF), Rating: 4.0, Sale Price: 1599
Product: L'Oreal Paris Volume Million Lashes Waterproof Mascara, Rating: 4.0, Sale Price: 999
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 5.0
Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best R

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/nyka_top_brands_cosmetics_product_reviews.csv')

# Convert sale_price to numeric if necessary
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Ensure rating is numeric
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['price', 'review_rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization
n_qubits = 3
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['review_rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['review_rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['review_rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['review_rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='review_rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='review_rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 4.0
Top 5 Recommended Products:
Product: Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF), Rating: 4.0, Sale Price: 1599
Product: L'Oreal Paris Volume Million Lashes Waterproof Mascara, Rating: 4.0, Sale Price: 999
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 5.0
Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best R

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/nyka_top_brands_cosmetics_product_reviews.csv')

# Convert sale_price to numeric if necessary
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Ensure rating is numeric
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['price', 'review_rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['review_rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['review_rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['review_rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['review_rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='review_rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='review_rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 4.0
Top 5 Recommended Products:
Product: Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF), Rating: 4.0, Sale Price: 1599
Product: L'Oreal Paris Volume Million Lashes Waterproof Mascara, Rating: 4.0, Sale Price: 999
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 4.8
Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best R

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/nyka_top_brands_cosmetics_product_reviews.csv')

# Convert sale_price to numeric if necessary
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Ensure rating is numeric
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['price', 'review_rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization
n_qubits = 5
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['review_rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['review_rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['review_rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['review_rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='review_rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='review_rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

<ipython-input-17-aefee5edc0db>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_sales'] = df['price']


Please enter the minimum rating (0-10): 4.0
Top 5 Recommended Products:
Product: Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF), Rating: 4.0, Sale Price: 1599
Product: L'Oreal Paris Volume Million Lashes Waterproof Mascara, Rating: 4.0, Sale Price: 999
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 5.0
Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best R

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/nyka_top_brands_cosmetics_product_reviews.csv')

# Convert sale_price to numeric if necessary
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Ensure rating is numeric
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['price', 'review_rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization
n_qubits = 6
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['review_rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['review_rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['review_rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['review_rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='review_rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='review_rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 4.0
Top 5 Recommended Products:
Product: Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF), Rating: 4.0, Sale Price: 1599
Product: L'Oreal Paris Volume Million Lashes Waterproof Mascara, Rating: 4.0, Sale Price: 999
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 5.0
Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best R

In [ ]:
import pandas as pd
import numpy as np
import pennylane as qml
import time

# Load the data
df = pd.read_csv('/content/nyka_top_brands_cosmetics_product_reviews.csv')

# Convert sale_price to numeric if necessary
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# Ensure rating is numeric
df['review_rating'] = pd.to_numeric(df['review_rating'], errors='coerce')

# Drop rows with missing sale_price or rating
df = df.dropna(subset=['price', 'review_rating'])

# Calculate total sales for each product (assuming each row represents a single sale)
df['total_sales'] = df['price']

# Prompt the user for minimum rating
user_rating = float(input("Please enter the minimum rating (0-10): "))

start = time.time()

# Quantum device initialization
n_qubits = 7
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def circuit(params):
    # Apply Hadamard gates to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply controlled-phase gates based on parameters to introduce correlation
    for i in range(n_qubits - 1):
        qml.CRot(params[i], params[i + 1], params[i] + params[i + 1], wires=[i, i + 1])

    # Apply controlled-Z gates for entanglement
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i + 1])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def quantum_objective(params):
    return np.sum(circuit(params))

# Define the Quantum-Inspired Bee Algorithm
class QuantumBeeInspiredAlgorithm:
    def __init__(self, data, num_scouts=10, num_best_sites=5, num_perspective_sites=3, num_iterations=50):
        self.data = data
        self.num_scouts = min(num_scouts, len(data))  # Adjust number of scouts if data is small
        self.num_best_sites = min(num_best_sites, self.num_scouts)
        self.num_perspective_sites = num_perspective_sites
        self.num_iterations = num_iterations

    def search(self):
        best_solution = None
        best_rating = -np.inf

        for iteration in range(self.num_iterations):
            scouts = self.scout_bees()
            if not scouts:
                print("No valid scout bees generated.")
                break

            best_sites = self.select_best_sites(scouts)

            for site in best_sites:
                solution, rating = self.local_search(site)
                if rating > best_rating:
                    best_solution = solution
                    best_rating = rating

            print(f"Iteration {iteration + 1}/{self.num_iterations}, Best Rating: {best_rating}")

        return best_solution, best_rating

    def scout_bees(self):
        scouts = []
        max_index = len(self.data) - 1
        if max_index < 4:
            print("Not enough products to generate scouts.")
            return scouts

        for _ in range(self.num_scouts):
            product_indices = np.random.choice(range(max_index + 1), 5, replace=False)
            scout_solution = self.data.iloc[product_indices]
            scouts.append(scout_solution)
        return scouts

    def select_best_sites(self, scouts):
        if not scouts:
            print("No scouts to select from.")
            return []

        scouts = sorted(scouts, key=lambda x: x['review_rating'].mean(), reverse=True)
        return scouts[:self.num_best_sites]

    def local_search(self, site):
        best_local_solution = site
        best_local_rating = site['review_rating'].mean()

        for _ in range(self.num_perspective_sites):
            new_solution = self.quantum_optimize(site)
            new_rating = new_solution['review_rating'].mean()
            if new_rating > best_local_rating:
                best_local_solution = new_solution
                best_local_rating = new_rating

        return best_local_solution, best_local_rating

    def quantum_optimize(self, site):
        params = np.random.rand(n_qubits)
        opt = qml.GradientDescentOptimizer(stepsize=0.1)
        for _ in range(10):  # Quantum optimization steps
            params = opt.step(quantum_objective, params)
        optimized_indices = np.argsort(params)[:min(5, len(site))]  # Select top indices based on quantum optimization
        optimized_indices = np.clip(optimized_indices, 0, len(site) - 1)  # Ensure indices are within bounds
        return site.iloc[optimized_indices]

# Filter the dataset based on user input
filtered_df = df[df['review_rating'] == user_rating]

# If the filtered dataset is empty, print a message and exit
if filtered_df.empty:
    print("No products match the given criteria.")
else:
    # Sort the filtered dataset by rating in descending order
    sorted_df = filtered_df.sort_values(by='review_rating', ascending=False)

    # Get the top 5 recommended products
    top_5_products = sorted_df.head(5)

    # Print the top 5 recommended products
    print("Top 5 Recommended Products:")
    for index, row in top_5_products.iterrows():
        print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

# Usage
algorithm = QuantumBeeInspiredAlgorithm(df)
best_solution, best_rating = algorithm.search()

# Ensure the best solution contains exactly 5 products, if possible
if len(best_solution) < 5:
    remaining_products = df[~df.index.isin(best_solution.index)]
    additional_products = remaining_products.sort_values(by='review_rating', ascending=False).head(5 - len(best_solution))
    best_solution = pd.concat([best_solution, additional_products])

print(f"Best solution found: {best_solution}, Best rating: {best_rating}")

print("Top 5 Recommended Products from Quantum Bee Algorithm:")
for index, row in best_solution.iterrows():
    print(f"Product: {row['product_title']}, Rating: {row['review_rating']}, Sale Price: {row['price']}")

print("Time Consumed")
print("% s seconds" % (time.time() - start))

Please enter the minimum rating (0-10): 4.0
Top 5 Recommended Products:
Product: Olay Ultra Lightweight Moisturiser: Luminous Whip Day Cream (non SPF), Rating: 4.0, Sale Price: 1599
Product: L'Oreal Paris Volume Million Lashes Waterproof Mascara, Rating: 4.0, Sale Price: 999
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509
Product: L'Oreal Paris Le Bar Powder Blush - 8 City Explorer, Rating: 4.0, Sale Price: 509


/usr/local/lib/python3.10/dist-packages/pennylane/_grad.py:216: UserWarning: Attempted to differentiate a function with no trainable parameters. If this is unintended, please add trainable parameters via the 'requires_grad' attribute or 'argnum' keyword.
  warnings.warn(


Iteration 1/50, Best Rating: 5.0
Iteration 2/50, Best Rating: 5.0
Iteration 3/50, Best Rating: 5.0
Iteration 4/50, Best Rating: 5.0
Iteration 5/50, Best Rating: 5.0
Iteration 6/50, Best Rating: 5.0
Iteration 7/50, Best Rating: 5.0
Iteration 8/50, Best Rating: 5.0
Iteration 9/50, Best Rating: 5.0
Iteration 10/50, Best Rating: 5.0
Iteration 11/50, Best Rating: 5.0
Iteration 12/50, Best Rating: 5.0
Iteration 13/50, Best Rating: 5.0
Iteration 14/50, Best Rating: 5.0
Iteration 15/50, Best Rating: 5.0
Iteration 16/50, Best Rating: 5.0
Iteration 17/50, Best Rating: 5.0
Iteration 18/50, Best Rating: 5.0
Iteration 19/50, Best Rating: 5.0
Iteration 20/50, Best Rating: 5.0
Iteration 21/50, Best Rating: 5.0
Iteration 22/50, Best Rating: 5.0
Iteration 23/50, Best Rating: 5.0
Iteration 24/50, Best Rating: 5.0
Iteration 25/50, Best Rating: 5.0
Iteration 26/50, Best Rating: 5.0
Iteration 27/50, Best Rating: 5.0
Iteration 28/50, Best Rating: 5.0
Iteration 29/50, Best Rating: 5.0
Iteration 30/50, Best R

#end